## **Contoh: Ejen Multi-AI untuk Tempahan Hotel**

Dalam dunia yang serba pantas hari ini, merancang perjalanan perniagaan melibatkan lebih daripada sekadar menempah penerbangan dan bilik hotel. Ia memerlukan tahap koordinasi dan kecekapan yang kadangkala sukar dicapai. Di sinilah Ejen Multi-AI memainkan peranan, merevolusikan cara kita mengurus keperluan perjalanan.

Bayangkan mempunyai pasukan ejen pintar yang tersedia, bekerjasama untuk mengurus setiap aspek perjalanan anda dengan ketepatan dan kemudahan. Dengan teknologi AI canggih kami, kami telah mencipta ejen khusus untuk perkhidmatan tempahan dan pengaturan jadual perjalanan, memastikan pengalaman perjalanan yang lancar dan bebas tekanan.

Ini adalah senario asas. Apabila merancang perjalanan perniagaan, kita perlu berunding dengan ejen perjalanan perniagaan untuk mendapatkan maklumat tiket penerbangan, maklumat hotel, dan sebagainya. Melalui Ejen AI, kita boleh membina ejen untuk perkhidmatan tempahan dan ejen untuk pengaturan jadual perjalanan supaya dapat bekerjasama dan meningkatkan tahap kecerdasan.


# Memulakan Perkhidmatan Azure AI Agent dan mendapatkan maklumat konfigurasi daripada **.env**

### **.env**

Buat fail .env

**.env** mengandungi string sambungan Perkhidmatan Azure AI Agent, model yang digunakan oleh AOAI, dan perkhidmatan API Carian Google yang berkaitan, ENDPOINT, dan lain-lain.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nama Penggunaan Model Perkhidmatan Azure AI Agent Anda"

[**NOTE**] Anda memerlukan model dengan Had Kadar 100,000 (Token per minit) dan Had Kadar 600 (Permintaan per minit).

  Anda boleh mendapatkan model di Azure AI Foundry - Model dan Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "String Sambungan Projek Perkhidmatan Azure AI Agent Anda"

  Anda boleh mendapatkan string sambungan projek di paparan keseluruhan projek anda di Skrin Portal AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Kunci API Carian SERPAPI Anda"
- **SERPAPI_SEARCH_ENDPOINT** = "Endpoint Carian SERPAPI Anda"

Untuk mendapatkan Nama Penggunaan Model dan String Sambungan Projek Perkhidmatan Azure AI Agent, anda perlu mencipta Perkhidmatan Azure AI Agent. Adalah disarankan untuk menggunakan [templat ini](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) untuk menciptanya secara langsung (***Nota:*** Perkhidmatan Azure AI Agent kini ditetapkan di kawasan terhad. Adalah disarankan agar anda merujuk kepada [pautan ini](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) untuk menetapkan kawasan).

Ejen perlu mengakses SERPAPI. Adalah disarankan untuk mendaftar menggunakan [pautan ini](https://serpapi.com/searches). Selepas pendaftaran, anda boleh mendapatkan API KEY dan ENDPOINT unik.


# Log Masuk ke Azure

Anda kini perlu log masuk ke Azure. Buka terminal dalam VScode dan jalankan arahan `az login`.


# Persediaan

Untuk menjalankan notebook ini, anda perlu memasang perpustakaan berikut. Berikut adalah senarai perpustakaan yang diperlukan dan arahan pemasangan pip yang berkaitan:

azure-identity: Untuk pengesahan Azure.  
requests: Untuk membuat permintaan HTTP.  
semantic-kernel: Untuk rangka kerja kernel semantik (mengandaikan ini adalah perpustakaan khusus atau spesifik, anda mungkin perlu memasangnya dari sumber atau repositori tertentu).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Penjelasan:
import asyncio: Ini mengimport modul asyncio, yang menyediakan sokongan untuk pengaturcaraan tak segerak dalam Python. Ia membolehkan anda menulis kod serentak menggunakan sintaks async dan await.  
from typing: Annotated: Ini mengimport jenis Annotated daripada modul typing. Annotated digunakan untuk menambah metadata kepada petunjuk jenis, yang boleh berguna untuk pelbagai tujuan seperti pengesahan, dokumentasi, atau alat bantu.


In [ ]:
import asyncio,os
from typing import Annotated

# Penjelasan:
Dengan menggunakan from dotenv import load_dotenv dan load_dotenv(), anda boleh menguruskan tetapan konfigurasi dan maklumat sensitif (seperti kunci API dan URL pangkalan data) dengan mudah dalam fail .env, memastikan ia terpisah daripada kod sumber anda dan menjadikan aplikasi anda lebih selamat serta lebih mudah untuk dikonfigurasi.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Penjelasan:

Pernyataan Import: from azure.identity.aio import DefaultAzureCredential: Ini mengimport kelas DefaultAzureCredential dari modul azure.identity.aio. Bahagian aio dalam nama modul menunjukkan bahawa ia direka untuk operasi asinkron.

Tujuan DefaultAzureCredential: Kelas DefaultAzureCredential adalah sebahagian daripada Azure SDK untuk Python. Ia menyediakan cara lalai untuk mengesahkan dengan perkhidmatan Azure. Ia cuba mengesahkan menggunakan pelbagai kaedah dalam urutan tertentu, seperti pembolehubah persekitaran, identiti terurus, dan kelayakan Azure CLI.

Operasi Asinkron: Modul aio menunjukkan bahawa kelas DefaultAzureCredential menyokong operasi asinkron. Ini bermaksud anda boleh menggunakannya dengan asyncio untuk melakukan permintaan pengesahan tanpa menyekat.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Penjelasan:
Mengimport pelbagai modul dan kelas daripada pakej semantic_kernel. Berikut adalah perincian setiap import:

AgentGroupChat daripada semantic_kernel.agents: Kelas ini mengendalikan fungsi berkaitan sembang kumpulan untuk ejen AI. AzureAIAgent dan AzureAIAgentSettings daripada semantic_kernel.agents.azure_ai

AzureAIAgent: Kelas ini digunakan untuk mencipta dan mengurus ejen AI yang menggunakan perkhidmatan Azure AI.

AzureAIAgentSettings: Kelas ini digunakan untuk mengkonfigurasi tetapan bagi AzureAIAgent. TerminationStrategy daripada semantic_kernel.agents.strategies.termination.termination_strategy:

Kelas ini mentakrifkan strategi untuk menamatkan pelaksanaan ejen AI di bawah keadaan tertentu. ChatMessageContent daripada semantic_kernel.contents.chat_message_content:

Kelas ini digunakan untuk mengendalikan kandungan mesej sembang.
AuthorRole daripada semantic_kernel.contents.utils.author_role:

Kelas ini mentakrifkan peranan yang berbeza untuk pengarang dalam konteks mesej sembang.

kernel_function daripada semantic_kernel.functions.kernel_function_decorator: Penghias ini digunakan untuk mentakrifkan fungsi kernel, iaitu fungsi yang boleh dilaksanakan dalam rangka kerja semantic kernel.
Import-import ini menyediakan komponen yang diperlukan untuk mencipta dan mengurus ejen AI yang boleh berinteraksi dalam persekitaran sembang kumpulan, mungkin untuk tugas seperti menempah hotel atau aktiviti serupa.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Penjelasan:
Seterusnya, kita mengimport kelas CodeInterpreterTool daripada modul azure.ai.projects.models.

CodeInterpreterTool: Kelas ini adalah sebahagian daripada Azure AI SDK dan digunakan untuk mentafsir serta melaksanakan kod dalam konteks projek AI. Ia menyediakan fungsi untuk menjalankan potongan kod, menganalisis kod, atau mengintegrasikan pelaksanaan kod dalam aliran kerja AI. 
Import ini menyediakan komponen yang diperlukan untuk menggunakan CodeInterpreterTool dalam projek anda, yang boleh berguna untuk tugas-tugas yang melibatkan tafsiran dan pelaksanaan kod secara dinamik.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Penjelasan: 
Kelas ApprovalTerminationStrategy menyediakan strategi khusus untuk menghentikan operasi ejen AI. Ejen tersebut akan berhenti jika mesej terakhir dalam sejarah interaksinya mengandungi perkataan "saved". Ini boleh berguna dalam situasi di mana tugas ejen dianggap selesai setelah ia menerima pengesahan bahawa sesuatu telah "saved". Tetapkan kaedah interaksi. Selepas pelan tempahan disimpan, ia boleh dihentikan apabila menerima isyarat saved.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Penjelasan:

Baris kod ini memulakan objek AzureAIAgentSettings dengan tetapan lalai atau yang telah ditentukan dengan memanggil kaedah create(). Objek tetapan ini (ai_agent_settings) kemudian boleh digunakan untuk mengkonfigurasi dan menguruskan instans AzureAIAgent.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Penjelasan:
Dengan mengimport perpustakaan requests, anda boleh membuat permintaan HTTP dengan mudah dan berinteraksi dengan perkhidmatan web dalam kod Python anda.


In [ ]:
import requests

# Penjelasan:
Ini adalah pemboleh ubah yang menyimpan kunci API untuk mengakses perkhidmatan API SERP (Search Engine Results Page). Kunci API adalah pengenal unik yang digunakan untuk mengesahkan permintaan yang dikaitkan dengan akaun anda.

'GOOGLE_SEARCH_API_KEY': Ini adalah rentetan pengganti. Anda perlu menggantikan 'GOOGLE_SEARCH_API_KEY' dengan kunci API SERP sebenar anda.

Tujuan: Tujuan baris ini adalah untuk menyimpan kunci API dalam pemboleh ubah supaya ia boleh digunakan untuk mengesahkan permintaan kepada perkhidmatan API SERP. Kunci API diperlukan untuk mengakses perkhidmatan dan melakukan carian.

Cara Mendapatkan Kunci API SERP: Untuk mendapatkan kunci API SERP, ikuti langkah-langkah umum berikut di https://serpapi.com (langkah-langkah tepat mungkin berbeza bergantung pada perkhidmatan API SERP yang anda gunakan):

Pilih Perkhidmatan API SERP: Terdapat beberapa perkhidmatan API SERP yang tersedia, seperti SerpAPI, Google Custom Search JSON API, dan lain-lain. Pilih yang paling sesuai dengan keperluan anda.

Daftar Akaun:

Pergi ke laman web perkhidmatan API SERP yang dipilih https://www.serpapi.com dan daftar akaun. Anda mungkin perlu memberikan beberapa maklumat asas dan mengesahkan alamat e-mel anda.

Cipta Kunci API:

Selepas mendaftar, log masuk ke akaun anda dan navigasi ke bahagian API atau papan pemuka. Cari pilihan untuk mencipta atau menjana kunci API baharu.
Salin Kunci API:

Setelah kunci API dijana, salin kunci tersebut. Kunci ini akan digunakan untuk mengesahkan permintaan anda kepada perkhidmatan API SERP.
Gantikan Pengganti:

Gantikan pengganti dalam fail .env anda.


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Penjelasan:
Kelas BookingPlugin menyediakan kaedah untuk menempah hotel dan penerbangan menggunakan API Carian Google dari Serpapi.com. Ia membina parameter yang diperlukan, menghantar permintaan API, dan memproses respons untuk mengembalikan maklumat tempahan yang berkaitan. Kunci API (SERPAPI_SEARCH_API_KEY) dan titik akhir (SERPAPI_SEARCH_ENDPOINT) digunakan untuk mengesahkan dan menghantar permintaan ke API Carian Google.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Penjelasan:
Kelas SavePlugin menyediakan kaedah saving_plan untuk menyimpan rancangan perjalanan menggunakan perkhidmatan Azure AI. Ia menyediakan kelayakan Azure, mencipta ejen AI, memproses input pengguna untuk menghasilkan dan menyimpan kandungan rancangan perjalanan, serta mengurus operasi penyimpanan fail dan pembersihan. Kaedah ini mengembalikan "Saved" apabila selesai dengan jayanya.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Penjelasan:
Kod ini menyediakan ejen Azure AI untuk menguruskan tempahan penerbangan dan hotel, serta menyimpan rancangan perjalanan berdasarkan input pengguna. Ia menggunakan kelayakan Azure untuk mencipta dan mengkonfigurasi ejen, memproses input pengguna melalui sembang kumpulan, dan memastikan pembersihan yang betul selepas tugas diselesaikan. Ejen-ejen ini menggunakan plugin tertentu (BookingPlugin dan SavePlugin) untuk melaksanakan tugas masing-masing.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
